# Problem 3

Before you start to work on Problem 3, please visit [this page](https://canvas.mit.edu/files/1128379/download?download_frd=1) to download the installation instructions.

Please install Julia (v1.5.3), JuMP (v0.20.0) and Gurobi (v9.1.1) on your computer as instructed by the tutorial on Canvas. These are the versions that the teaching team will provide support for. You should use Gurobi as the solver for all JuMP related problems, and use the Plots or StatsPlots packages for graphing.

Before you submit this Jupyter notebook, please use (1) `Kernel` $\rightarrow$ `Restart & Run All`, and (2) `File` $\rightarrow$ `Download a` $\rightarrow$ `HMTL (.html)` on the notebook menu to make the output of your codes available to the TAs.



## Class Assignments

The Salanter Akiba Riverdale (SAR) Academy is a coeducational, private Modern Orthodox Jewish day school located in New York City. Ever summer, the SAR Academy must create class assignments for their elementary school students. Each grade of 80-100 students must be divided into different classes. Requests for assignments are made by parents, teachers, and school therapists. These requests include pairs of students that should be placed together, pairs of students that should not be placed together, and requests for students to be placed in classes that better suit their unique or special academic needs. These requests often conflict with each other, and it falls on the administration to prioritize which requests should be fulfilled over others.

In this exercise, we will solve a simplified version of the problem faced by the SAR Academy with 40 students and two classes. (The full optimization problem is currently being used to assist administrators at the SAR Academy.) The parents or guardians of each of the 40 students are asked to submit preferences for class 1 or class 2. These preferences often depend on the teaching style of the teachers, the teachers older siblings have had in the past, and characteristics of the class (one class is called an "inclusion class," which is better for students with academic needs). The parents give a ranking of 1 to the class they prefer (their first choice), and a ranking of 2 to their second choice. This data, as well as the gender of each of the students, is given in the spreadsheet `ClassAssignments.csv`.   When solving this problem, please use the prepared Jupyter notebook.

## Part (a)

The  problem  faced  by  the  SAR  Academy  is  to  decide  which  students  should  be assigned to which classes,  to satisfy as many of the parent preferences as possible (note thatsince  smaller  numbers  for  the  preferences  are  better,  this  is  a  minimization  problem).   Each student must be assigned to exactly one class, and there should be exactly 20 students in eachclass.

### (i)

Formulate this problem as an integer optimization problem. Describe the decision variables, objective function, and constraints.

On your PDF writeup, write down the formulation of this problem. Be sure to clearly indicate what your decision variables, objective, and constraints are.


Let's work on some Julia code now. First, import the dataset from the spreadsheet `ClassAssignments.csv`. You can do it in the cell below.

In [1]:
## if you have not installed package "CSV", please uncomment the line below and execute it
# using Pkg; Pkg.add("CSV")
using DataFrames, CSV
portfolio = CSV.read("ClassAssignments.csv", DataFrame)

,Student Number,Parent Preference for Class 1,Parent Preference for Class 2,Male or Female? (M or F)
,Int64,Int64,Int64,String
1,1,1,2,M
2,2,1,2,M
3,3,2,1,M
4,4,1,2,M
5,5,1,2,M
6,6,2,1,M
7,7,1,2,M
8,8,2,1,M
9,9,1,2,M


Then, in the cell below, write Julia/JuMP codes to describe the program.

In [2]:
#--- Model specification
using JuMP, DataFrames, Gurobi


classAssignment = Model(with_optimizer(Gurobi.Optimizer))

p1 = portfolio[!, 2]
p2 = portfolio[!, 3]

#Decision Variables
@variables classAssignment begin
    x1[1:40], Bin
    x2[1:40], Bin
end

#Objective Function
@objective(classAssignment, Min, sum( (p1[i]*x1[i]) + (p2[i]*x2[i]) for i=1:40))
#@objective(m,Min,sum{c[i,j]*x[i,j],i=1:5,j=1:3})

#Constraints
@constraint(classAssignment, class1_cap, sum(x1[i] for i=1:40)==20)
@constraint(classAssignment, class2_cap, sum(x2[i] for i=1:40)==20)
for i in 1:40
    @constraint(classAssignment, x1[i]+x2[i] == 1)
end

classAssignment



Academic license - for non-commercial use only - expires 2021-04-30


A JuMP Model
Minimization problem with:
Variables: 80
Objective function type: GenericAffExpr{Float64,VariableRef}
`GenericAffExpr{Float64,VariableRef}`-in-`MathOptInterface.EqualTo{Float64}`: 42 constraints
`VariableRef`-in-`MathOptInterface.ZeroOne`: 80 constraints
Model mode: AUTOMATIC
CachingOptimizer state: EMPTY_OPTIMIZER
Solver name: Gurobi
Names registered in the model: class1_cap, class2_cap, x1, x2

### (ii)

In the cell below, write Julia/JuMP codes to solve this problem with Gurobi.

In [3]:
#--- Write codes here to solve this problem with Gurobi
optimize!(classAssignment)

@show objective_value(classAssignment)




Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (mac64)
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 42 rows, 80 columns and 160 nonzeros
Model fingerprint: 0x58ac72eb
Variable types: 0 continuous, 80 integer (80 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 2e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 2e+01]
Found heuristic solution: objective 60.0000000
Presolve removed 42 rows and 80 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.00 seconds
Thread count was 1 (of 8 available processors)

Solution count 2: 42 

Optimal solution found (tolerance 1.00e-04)
Best objective 4.200000000000e+01, best bound 4.200000000000e+01, gap 0.0000%

User-callback calls 35, time in user-callback 0.00 sec
objective_value(classAssignment) = 42.0


42.0

What is the optimal solution?  Give the values of the decision variables as well as the value of the optimal objective function value in the below cell.

In [4]:
#--- Write codes here to print your solutions

list = JuMP.value.(x1[1:40])
list2 = JuMP.value.(x2[1:40])
for i in 1:40
    println("x with j=1, x= "* string(i)*": "*string(list[i]))
    println("x with j=2, x= "* string(i)*": "*string(list2[i]))
end 
println("Final Value: "*string(objective_value(classAssignment)))





x with j=1, x= 1: 1.0
x with j=2, x= 1: 0.0
x with j=1, x= 2: 1.0
x with j=2, x= 2: 0.0
x with j=1, x= 3: 0.0
x with j=2, x= 3: 1.0
x with j=1, x= 4: 1.0
x with j=2, x= 4: 0.0
x with j=1, x= 5: 1.0
x with j=2, x= 5: 0.0
x with j=1, x= 6: 0.0
x with j=2, x= 6: 1.0
x with j=1, x= 7: 1.0
x with j=2, x= 7: 0.0
x with j=1, x= 8: 0.0
x with j=2, x= 8: 1.0
x with j=1, x= 9: 1.0
x with j=2, x= 9: 0.0
x with j=1, x= 10: 1.0
x with j=2, x= 10: 0.0
x with j=1, x= 11: 1.0
x with j=2, x= 11: 0.0
x with j=1, x= 12: 0.0
x with j=2, x= 12: 1.0
x with j=1, x= 13: 1.0
x with j=2, x= 13: 0.0
x with j=1, x= 14: 1.0
x with j=2, x= 14: 0.0
x with j=1, x= 15: 1.0
x with j=2, x= 15: 0.0
x with j=1, x= 16: 0.0
x with j=2, x= 16: 1.0
x with j=1, x= 17: 1.0
x with j=2, x= 17: 0.0
x with j=1, x= 18: 1.0
x with j=2, x= 18: 0.0
x with j=1, x= 19: 1.0
x with j=2, x= 19: 0.0
x with j=1, x= 20: 1.0
x with j=2, x= 20: 0.0
x with j=1, x= 21: 0.0
x with j=2, x= 21: 1.0
x with j=1, x= 22: 1.0
x with j=2, x= 22: 0.0
x with

### (iii)

How  many  students  received  their  first  choice  class,  according  to  the  parent preferences?   (HINT:  You  should  be  able  to  answer  this  by  just  looking  at  the  optimal objective function value.) Write your solution in the Markdown box below.

*38 students received their first choice class, 2 students received their second choice.*

## Part (b)

After looking at the optimal solution, the SAR academy decided that they would like to adjust the formulation for a better boy/ girl ratio in the classes. They would like to limit the boys in each class to no more than 12.

### (i)

What constraint(s) do you need to add to your model to incorporate this adjustment? Write down your solution on the PDF writeup.

### (ii)

Add the necessary constraints and resolve your model. In the cell below, write Julia/JuMP codes to solve this problem with Gurobi.

In [5]:
#--- Write codes here to solve this problem with Gurobi
#--- Model specification
using JuMP, DataFrames, Gurobi

genders = portfolio[!, 4]

function genderConversion(gender)
    result = []
    for each in genders
        if each == "M"
            push!(result, 1)
        else 
            push!(result, 0)
        end
    end
    return result
end

classAssignment = Model(with_optimizer(Gurobi.Optimizer))

p1 = portfolio[!, 2]
p2 = portfolio[!, 3]
genderList = genderConversion(genders)

#Decision Variables
@variables classAssignment begin
    x1[1:40], Bin
    x2[1:40], Bin
end

#Objective Function
@objective(classAssignment, Min, sum( (p1[i]*x1[i]) + (p2[i]*x2[i]) for i=1:40))
#@objective(m,Min,sum{c[i,j]*x[i,j],i=1:5,j=1:3})

#Constraints
@constraint(classAssignment, class1_cap, sum(x1[i] for i=1:40)==20)
@constraint(classAssignment, class2_cap, sum(x2[i] for i=1:40)==20)
@constraint(classAssignment, class1_boys_cap, sum(x1[i]*genderList[i] for i=1:40)<= 12)
@constraint(classAssignment, class2_boys_cap, sum(x2[i]*genderList[i] for i=1:40)<= 12)


for i in 1:40
    @constraint(classAssignment, x1[i]+x2[i] == 1)
end



classAssignment





Academic license - for non-commercial use only - expires 2021-04-30


A JuMP Model
Minimization problem with:
Variables: 80
Objective function type: GenericAffExpr{Float64,VariableRef}
`GenericAffExpr{Float64,VariableRef}`-in-`MathOptInterface.EqualTo{Float64}`: 42 constraints
`GenericAffExpr{Float64,VariableRef}`-in-`MathOptInterface.LessThan{Float64}`: 2 constraints
`VariableRef`-in-`MathOptInterface.ZeroOne`: 80 constraints
Model mode: AUTOMATIC
CachingOptimizer state: EMPTY_OPTIMIZER
Solver name: Gurobi
Names registered in the model: class1_boys_cap, class1_cap, class2_boys_cap, class2_cap, x1, x2

What is the optimal solution now? Give the values of the decision variables as well as the value of the optimal objective function value in the below cell.

In [6]:
#--- Write codes here to print your solutions
optimize!(classAssignment)

@show objective_value(classAssignment)

list = JuMP.value.(x1[1:40])
list2 = JuMP.value.(x2[1:40])
for i in 1:40
    println("x with j=1, x= "* string(i)*": "*string(list[i]))
    println("x with j=2, x= "* string(i)*": "*string(list2[i]))
end 
println("Final Value: "*string(objective_value(classAssignment)))





Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (mac64)
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 44 rows, 80 columns and 206 nonzeros
Model fingerprint: 0x5f0aa283
Variable types: 0 continuous, 80 integer (80 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 2e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 2e+01]
Found heuristic solution: objective 60.0000000
Presolve removed 44 rows and 80 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.00 seconds
Thread count was 1 (of 8 available processors)

Solution count 2: 46 

Optimal solution found (tolerance 1.00e-04)
Best objective 4.600000000000e+01, best bound 4.600000000000e+01, gap 0.0000%

User-callback calls 44, time in user-callback 0.00 sec
objective_value(classAssignment) = 46.0
x with j=1, x= 1: 1.0
x with j=2, x= 1: 0.0
x with j=1, x= 2: 1.0
x with j

How does it compare to the previous solution? Discuss this in the Markdown box below.

*This solution is worse because 4 more students go their second option (in the previous solution, 2 students got their second option but in this solution 6 did). *

### (iii)

How many students received their first choice class in this new solution, according to the parent preferences? Write down your solution in the Markdown box below.

*34 students got their first choice according to parent preferences. *

## Part (c)

Now, we will add some logical constraints to capture additional
preferences of parents, teachers, and school therapists.

### (i)

Students 10 and 11 are twins, and the school has a policy that twins must be placed in different classes. What constraint(s) needs to be added to the model to implement this policy? Write down your solution on the PDF writeup.

### (ii)

Students 4, 9, 15, 25, 30, and 36 are all from the same neighborhood. The school would like to put at least 2 students from this neighborhood in each class. What constraint(s) needs to be added to the model to implement this policy? Write down your solution on the PDF writeup.

### (iii)

The school therapist strongly recommends that students 20 and 21 are placed in the same classroom, that student 1 is placed in classroom 2, and that student 40 is placed in classroom 2. What constraint(s) needs to be added to the model to implement this policy? Write down your solution on the PDF writeup.

### (iv)

Add all of these constraints to your model, and solve it again.

In the cell below, write Julia/JuMP codes to solve this new problem. Please use Gurobi as the solver.

In [7]:
#--- Write Julia/JuMP codes to solve this new problem with Gurob
using JuMP, DataFrames, Gurobi

genders = portfolio[!, 4]

function genderConversion(gender)
    result = []
    for each in genders
        if each == "M"
            push!(result, 1)
        else 
            push!(result, 0)
        end
    end
    return result
end

classAssignment = Model(with_optimizer(Gurobi.Optimizer))

p1 = portfolio[!, 2]
p2 = portfolio[!, 3]
genderList = genderConversion(genders)

#Decision Variables
@variables classAssignment begin
    x1[1:40], Bin
    x2[1:40], Bin
end

#Objective Function
@objective(classAssignment, Min, sum( (p1[i]*x1[i]) + (p2[i]*x2[i]) for i=1:40))
#@objective(m,Min,sum{c[i,j]*x[i,j],i=1:5,j=1:3})

#Constraints
@constraint(classAssignment, class1_cap, sum(x1[i] for i=1:40)==20)
@constraint(classAssignment, class2_cap, sum(x2[i] for i=1:40)==20)
@constraint(classAssignment, class1_boys_cap, sum(x1[i]*genderList[i] for i=1:40)<= 12)
@constraint(classAssignment, class2_boys_cap, sum(x2[i]*genderList[i] for i=1:40)<= 12)
@constraint(classAssignment, stud10_class1, x1[10]+x1[11]==1)
@constraint(classAssignment, stud10_class2, x2[10]+x2[11]==1)
@constraint(classAssignment, neighborhood_class1, x1[4]+x1[9]+x1[15]+x1[25]+x1[30]+x1[36]>=2)
@constraint(classAssignment, neighborhood_class2, x2[4]+x2[9]+x2[15]+x2[25]+x2[30]+x2[36]>=2)
@constraint(classAssignment, stud20, x1[20]==x1[21])
@constraint(classAssignment, stud1, x1[2]==1)
@constraint(classAssignment, stud40, x1[40]==1)


for i in 1:40
    @constraint(classAssignment, x1[i]+x2[i] == 1)
end



classAssignment





Academic license - for non-commercial use only - expires 2021-04-30


A JuMP Model
Minimization problem with:
Variables: 80
Objective function type: GenericAffExpr{Float64,VariableRef}
`GenericAffExpr{Float64,VariableRef}`-in-`MathOptInterface.EqualTo{Float64}`: 47 constraints
`GenericAffExpr{Float64,VariableRef}`-in-`MathOptInterface.GreaterThan{Float64}`: 2 constraints
`GenericAffExpr{Float64,VariableRef}`-in-`MathOptInterface.LessThan{Float64}`: 2 constraints
`VariableRef`-in-`MathOptInterface.ZeroOne`: 80 constraints
Model mode: AUTOMATIC
CachingOptimizer state: EMPTY_OPTIMIZER
Solver name: Gurobi
Names registered in the model: class1_boys_cap, class1_cap, class2_boys_cap, class2_cap, neighborhood_class1, neighborhood_class2, stud1, stud10_class1, stud10_class2, stud20, stud40, x1, x2

What is the optimal solution now? Give the values of the decision variables as well as the value of the optimal objective function value in the below cell.

In [8]:
#--- Write codes here to print your solutions

optimize!(classAssignment)

@show objective_value(classAssignment)

list = JuMP.value.(x1[1:40])
list2 = JuMP.value.(x2[1:40])
for i in 1:40
    println("x with j=1, x= "* string(i)*": "*string(list[i]))
    println("x with j=2, x= "* string(i)*": "*string(list2[i]))
end 
println("Final Value: "*string(objective_value(classAssignment)))





Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (mac64)
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 51 rows, 80 columns and 226 nonzeros
Model fingerprint: 0xba3fcb1d
Variable types: 0 continuous, 80 integer (80 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 2e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 2e+01]
Found heuristic solution: objective 58.0000000
Presolve removed 46 rows and 72 columns
Presolve time: 0.00s
Presolved: 5 rows, 8 columns, 22 nonzeros
Variable types: 0 continuous, 8 integer (2 binary)

Root relaxation: objective 4.600000e+01, 2 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0      46.0000000   46.00000  0.00%     -    0s

Explored 0 nodes (2 simplex iterations) in 0.00 seconds
Thread count 

How does the objective value compare to the previous solution, without all of these constraints? What does this tell us? Discuss this in the Markdown box below.

*This solution was the same as the previous solution without all of these extra constraints. This tells us that these constraints matched their parent preferences, so they ended up not affecting the final objective value.*